In [ ]:
# === ADVANCED FOREX BOT: MULTI-TIMEFRAME, AUTO-SELECTION, PROFIT-FIRST ===

import time
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import talib
from datetime import datetime, timedelta
import subprocess

# === CONFIG ===
SYMBOLS = [
    "XAUUSD", "EURUSD", "GBPUSD", "USDJPY", "AUDUSD",
    "USDCAD", "NZDUSD", "USDCHF", "EURJPY", "GBPJPY",
    "EURGBP", "CADJPY", "CHFJPY", "AUDJPY", "EURNZD"
]
TIMEFRAMES = [mt5.TIMEFRAME_M5, mt5.TIMEFRAME_M10, mt5.TIMEFRAME_M15]
LOT_SIZE = 0.01
MIN_PROFIT_USD = 0.10
MAX_TRADES = 5

# === INIT MT5 ===
subprocess.Popen(["C:/Program Files/Aone Ltd MetaTrader 5/terminal64.exe"], shell=True)
time.sleep(10)

if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

for symbol in SYMBOLS:
    info = mt5.symbol_info(symbol)
    if info and not info.visible:
        mt5.symbol_select(symbol, True)

# === GET PRICE DATA ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === HISTORICAL WEEKDAY BIAS ===
def historical_day_bias(symbol):
    start = datetime(2000, 1, 1)
    end = datetime.now()
    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, start, end)
    if rates is None or len(rates) == 0:
        return 0
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['weekday'] = df['time'].dt.weekday
    avg_diff = df.groupby('weekday')[['close', 'open']].apply(lambda x: (x['close'] - x['open']).mean())
    return avg_diff.get(datetime.now().weekday(), 0)

# === TRADE SIGNAL DECISION ===
def get_trade_signal(df, symbol):
    close = df['close'].astype(float).values
    open_ = df['open'].astype(float).values
    high = df['high'].astype(float).values
    low = df['low'].astype(float).values

    rsi = talib.RSI(close, timeperiod=14)
    ma = talib.MA(close, timeperiod=20)
    if len(rsi) == 0 or len(ma) == 0:
        return None

    patterns = {
        "engulfing": talib.CDLENGULFING(open_, high, low, close)[-1],
        "doji": talib.CDLDOJI(open_, high, low, close)[-1],
        "morning_star": talib.CDLMORNINGSTAR(open_, high, low, close)[-1],
        "hammer": talib.CDLHAMMER(open_, high, low, close)[-1]
    }

    last_rsi = rsi[-1]
    last_ma = ma[-1]
    last_price = close[-1]
    bullish_count = sum(1 for v in patterns.values() if v > 0)
    bearish_count = sum(1 for v in patterns.values() if v < 0)
    day_bias = historical_day_bias(symbol)

    print(f"{symbol} - RSI: {last_rsi:.2f}, MA: {last_ma:.5f}, Price: {last_price:.5f}, Bullish: {bullish_count}, Bearish: {bearish_count}, Bias: {day_bias:.5f}")

    if bullish_count >= 2 and last_rsi < 45 and last_price > last_ma and day_bias > 0:
        return "buy"
    elif bearish_count >= 2 and last_rsi > 60 and last_price < last_ma and day_bias < 0:
        return "sell"
    return None

# === EXECUTE TRADE ===
def open_trade(symbol, direction):
    tick = mt5.symbol_info_tick(symbol)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "auto_profit_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"✅ OPENED {direction.upper()} for {symbol} at {price}")
    else:
        print(f"❌ FAILED to open {symbol}: {result.retcode}")
        print(result._asdict())

# === CLOSE TRADES IN PROFIT ===
def check_and_close():
    for symbol in SYMBOLS:
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            pos = positions[0]
            profit = pos.profit
            print(f"{symbol} live profit: ${profit:.2f}")
            if profit >= MIN_PROFIT_USD:
                close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": pos.volume,
                    "type": close_type,
                    "position": pos.ticket,
                    "price": price,
                    "deviation": 20,
                    "magic": 10001,
                    "comment": "auto_exit_profit",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
                result = mt5.order_send(request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ CLOSED {symbol} with profit ${profit:.2f}")
                else:
                    print(f"❌ FAILED to close {symbol}: {result.retcode}")

# === MAIN LOOP ===
print("🚀 Running smart pattern+historical+indicator bot with high-confidence entries...")
try:
    while True:
        check_and_close()
        time.sleep(2)

        open_positions = mt5.positions_get()
        open_symbols = [pos.symbol for pos in open_positions] if open_positions else []

        for symbol in SYMBOLS:
            if open_symbols.count(symbol) == 0:
                for tf in TIMEFRAMES:
                    df = get_data(symbol, tf, 100)
                    if df is not None:
                        signal = get_trade_signal(df, symbol)
                        if signal:
                            open_trade(symbol, signal)
                            break
                time.sleep(1)
except KeyboardInterrupt:
    print("🛑 Bot manually stopped.")
finally:
    mt5.shutdown()


🚀 Running smart pattern+historical+indicator bot with high-confidence entries...
XAUUSD - RSI: 62.57, MA: 3309.78950, Price: 3314.93000, Bullish: 1, Bearish: 0, Bias: 0.79129
XAUUSD - RSI: 59.02, MA: 3309.06400, Price: 3314.89000, Bullish: 0, Bearish: 0, Bias: 0.79129
XAUUSD - RSI: 57.48, MA: 3306.85750, Price: 3314.89000, Bullish: 0, Bearish: 0, Bias: 0.79129
EURUSD - RSI: 71.52, MA: 1.13308, Price: 1.13471, Bullish: 0, Bearish: 0, Bias: 0.00019
EURUSD - RSI: 65.76, MA: 1.13285, Price: 1.13471, Bullish: 0, Bearish: 0, Bias: 0.00019
EURUSD - RSI: 65.93, MA: 1.13253, Price: 1.13471, Bullish: 0, Bearish: 0, Bias: 0.00019
GBPUSD - RSI: 56.67, MA: 1.34448, Price: 1.34474, Bullish: 0, Bearish: 0, Bias: 0.00000
GBPUSD - RSI: 51.78, MA: 1.34415, Price: 1.34389, Bullish: 0, Bearish: 0, Bias: 0.00000
GBPUSD - RSI: 53.23, MA: 1.34369, Price: 1.34376, Bullish: 0, Bearish: 0, Bias: 0.00000
USDJPY - RSI: 36.19, MA: 143.60325, Price: 143.42600, Bullish: 0, Bearish: 0, Bias: 0.00000
USDJPY - RSI: 41.